In [46]:
%reload_ext autoreload
%autoreload 2

import pathlib
from lifelines import CoxPHFitter
from lifelines.statistics import multivariate_logrank_test
import pandas as pd
from sksurv.ensemble import RandomSurvivalForest
from sksurv.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from typing import List

from tcga_brca_clinical import conf, io
import warnings
warnings.filterwarnings('ignore')



In [2]:
data = io.read_data(pathlib.Path("../data").joinpath(conf.PREPROCESSED_FILENAME))
data = data.sample(frac=1, random_state=0).reset_index(drop=True)
train_proportion = 0.8
cut = int(len(data)*train_proportion)

train_data = data[:cut]
test_data = data[cut:]

assert len(train_data) + len(test_data) == len(data)

## Feature selection

Let's pick some promising variables, run a Cox regression and analyse their concordance index, and finally picking the 3 largest.

In [3]:
features_ref = [
    "age_at_diagnosis_binned",
    "treatment",
    "race",
    "ajcc_pathologic_m",
    "ajcc_pathologic_stage",
]

concordance_index = {}

for feature in features_ref:
    data_to_fit = data[[feature] + ["survival_days", "observed_death"]]
    cph = CoxPHFitter()
    cph.fit(data_to_fit, duration_col="survival_days", event_col="observed_death", formula=feature)
    concordance_index[feature] = cph.concordance_index_

concordance_index = pd.DataFrame(list(concordance_index.items()), columns = ["feature", "concordance_index"]).sort_values(by="concordance_index", ascending=False)
concordance_index

,feature,concordance_index
4,ajcc_pathologic_stage,0.694512
1,treatment,0.639897
0,age_at_diagnosis_binned,0.639693
3,ajcc_pathologic_m,0.554623
2,race,0.514185


In [13]:
# features = concordance_index.nlargest(n=3, columns="concordance_index")["feature"].tolist()
features = ["ajcc_pathologic_stage", "treatment", "age_at_diagnosis_binned"]

## Cox regression model

In [53]:
cph = CoxPHFitter()
cph_train = train_data[features + ["survival_days", "observed_death"]]
cph_test = test_data[features + ["survival_days", "observed_death"]]

cph.fit(
    cph_train,
    duration_col="survival_days",
    event_col="observed_death",
    formula="+".join(features),
    show_progress=False,
    step_size=0.2
)

0.8058759913482336

In [ ]:
def categorize_features(data: pd.DataFrame) -> pd.DataFrame:
    cat_columns = data.select_dtypes(exclude="number").columns
    data_categorized = data.copy()
    data_categorized[cat_columns] = data[cat_columns].astype(dtype="category")
    return data_categorized

data_prep = categorize_features(data_prep)

## Random Survival Forest

In [49]:
def make_y_rsf(data: pd.DataFrame) -> np.ndarray:
    y_rsf = data[["observed_death", "survival_days"]]
    y_rsf["observed_death"] = y_rsf["observed_death"].astype(bool)
    y_rsf = y_rsf.to_records(index=False)
    return y_rsf

def make_X_rsf(data: pd.DataFrame, features: List[str]) -> pd.DataFrame:
    X_rsf = data[features]
    X_rsf = OneHotEncoder().fit_transform(X_rsf)
    return X_rsf


X_train = make_X_rsf(train_data, features)
X_test = make_X_rsf(test_data, features)

y_train = make_y_rsf(train_data)
y_test =  make_y_rsf(test_data)



rsf = RandomSurvivalForest(
    n_estimators=1000,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=conf.RANDOM_STATE,
)
rsf.fit(X_train, y_train)


0.772170151405912

## Evaluation

In [56]:
def print_score(model: str, method: str, score: float):
    print(f"{method} for model ({model}): {score}")

print_score("Cox’s proportional hazard model", "concordance index", cph.score(cph_test, scoring_method="concordance_index"))
print_score("Random Survival Forest", "concordance index", rsf.score(X_test, y_test))

concordance index for model (Cox’s proportional hazard model): 0.8058759913482336
concordance index for model (Random Survival Forest): 0.772170151405912
